# Homework 7

In the first class of the semester, you had to operate a reservoir for 6 seasons with the goal of having the highest end-period storage without overtopping. Here you'll formally optimize those operations! Your objective is to maximize the total storage level over the 6 seasons, subject to ensuring a $<1\%$ chance of overtopping. The reservoir has a capacity of $K=200$ units and a maximum release of $R_{max}=150$ units. The inflows, $Q$, are log-normally distributed with the log-space means $\mu_Y$ and standard deviations $\sigma_Y$ listed in Table 1, where $Y=\text{ln}(Q)$. The correlation in log-space flows between consecutive seasons is $\rho=0.5$.

| Parameter | Season 1 | Season 2 | Season 3 | Season 4 | Season 5 | Season 6 |  
|-----------|----------|----------|----------|----------|----------|----------|  
| $\mu_Y$ | 2.9 | 3.9 | 4.6 | 4.6 | 4.1 | 3.7 |
| $\sigma_Y$ | 0.5 | 0.4 | 0.3 | 0.4 | 0.3 | 0.2 |

$\color{red}{\text{The code below initializes these parameters. It is complete.}}$

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats as ss
from matplotlib import pyplot as plt
import matplotlib

# capacity of reservoir
K = 200

# maximum release
Rmax = 150

# parameters of random inflow
rho = 0.5 # lag-1 autocorrelation coefficient
muY = np.array([2.9, 3.9, 4.6, 4.6, 4.1, 3.7]) # average log-space flows
sigmaY = np.array([0.5, 0.4, 0.3, 0.4, 0.3, 0.2]) # standard deviation of log-space flows
meanQ = np.exp(muY + 0.5*sigmaY**2) # real-space mean flow
nSeasons = len(muY) # number of time steps

## Question 1: DDP (15 pts)

Write a function $\texttt{calcValueDDP}$ for discrete deterministic dynamic programming (DDP) that takes as input the following:

* $S$: 1-D array of storages representing the states
* $Q$: scalar of the mean inflow for the season whose releases are being optimized,
* $bounds$: 1-D array of length 2 with the lower and upper bounds on the release
* $FutureValue$: 1-D array of the future values (total future storages) associated with the states $S$ in the following season.

Have the function return the following:

* $Rbest$: 1-D array of optimal releases from each state in $S$
* $Vbest$: 1-D array of the present + future value associated with each state in $S$

$\color{red}{\text{Complete the code below.}}$

In [ ]:
def calcValueDDP(S, Q, bounds, FutureValue):



  return Rbest, Vbest

$\color{red}{\text{You can check that your calcValueDDP function returns the right values with the code below.}}$

In [ ]:
states = np.arange(0,201,20)
nStates = len(states)
DDP_FutureValue = np.zeros([nStates])
bounds = [0,Rmax]
R, DDP_FutureValue = calcValueDDP(states, meanQ[0], bounds, DDP_FutureValue)

print("R.mean() should equal [{}]. Yours equals [{}]".format(2.41, R.mean()))
print("FutureExpValue.mean() should equal [{}]. Yours equals [{}]".format(118.18, DDP_FutureValue.mean()))

Now use the following code chunk to perform discrete DDP with your $\texttt{calcValueDDP}$ function, solving for the optimal release each season from storage levels between 0 and 200 in increments of 20 assuming the mean inflow each season. $\color{red}{\text{The code below is complete.}}$

In [ ]:
# get indices of stages
nStages = len(meanQ)
forward_indices = np.arange(nStages)
backward_indices = forward_indices[::-1]
backward_indices = np.insert(backward_indices,0,0)

# discretize states
states = np.arange(0,201,20)
nStates = len(states)

# bounds on decision variables (releases)
# R is between 0 and R max each season
bounds = [0,Rmax]

# initialize matrices with costs of each state at each stage
# and optimal releases to make from each state at each stage
DDP_values = np.empty([nStates,nStages])
DDP_release_policy = np.empty([nStates,nStages])

# initialize FutureValue at 0 for all states; will update as we move backwards
DDP_FutureValue = np.zeros([nStates])

# begin backward-moving DP
loop = True
while loop:
  count = 0
  for index in backward_indices[0:-1]:
    # find optimal release and value of each state in this stage
    R, DDP_FutureValue = calcValueDDP(states, meanQ[index-1], bounds, DDP_FutureValue)

    # count iterations with no change in optimal release
    if np.all(R == DDP_release_policy[:,index-1]):
      count += 1

    # update best releases and value of each state if not yet in steady state
    DDP_values[:,index] = DDP_FutureValue
    DDP_release_policy[:,index-1] = R

  # stop loop if no change in optimal decisions across all iterations
  if count == len(backward_indices[0:-1]):
    break

DDP_release_policy_df = pd.DataFrame(DDP_release_policy,
                                 columns=["Season 1","Season 2","Season 3","Season 4","Season 5", "Season 6"],
                                 index=states)
DDP_release_policy_df.index.rename("Storage",inplace=True)
DDP_release_policy_df

$\color{red}{\text{Now copy the code above, but instead of assuming the mean inflow each season, pass the 99th percentile flow each season to $\texttt{calcValueDDP}$.}}$
$\color{red}{\text{Call your new variables DDP_values_99, DDP_release_policy_99, DDP_FutureValue_99 and DDP_release_policy_99_df.}}$  
$\color{red}{\text{Use the tolerance approach we used for SDP in class as a stopping criterion.}}$   
Note: Use $\texttt{np.nanmean}$ to find the average percent difference to ignore the cases in which the optimal release is 0 and the percent difference is therefore infinite.

In [ ]:
# find the 99th percentile flow each season
q_99 =

# re-run DDP using q_99 each season instead of meanQ
## NOTE: when calculating the average percent difference, use np.nanmean() rather than np.mean()


## Question 2: SDP Transition probabilities (10 pts)

Now find an optimal release policy using SDP. First, use the code below to discretize the inflows each season into 20 values between that season's 1st and 99th percentile, and then calculate the transition probabilities between them. $\color{red}{\text{The code below is complete.}}$

In [ ]:
forward_indices = np.append(forward_indices,0)

# find transition probabilities from discrete flow values in one season to the next
def calcTransProb(mu, sigma, rho, nLevels):
  transprob = np.empty([nLevels,nLevels])
  # discrete levels of log-space flows
  Ylevels1 = np.linspace(ss.norm.ppf(0.01,mu[0],sigma[0]),ss.norm.ppf(0.99,mu[0],sigma[0]),nLevels)
  Ylevels2 = np.linspace(ss.norm.ppf(0.01,mu[1],sigma[1]),ss.norm.ppf(0.99,mu[1],sigma[1]),nLevels)
  for i in range(nLevels):
    # find conditional distribution of Qlevels2[j] given flow is Qlevels1[i] in previous season
    mu_cond = mu[1] + rho*(sigma[1]/sigma[0])*(Ylevels1[i] - mu[0])
    sigma_cond = sigma[1] * np.sqrt((1-rho**2))
    for j in range(nLevels):
      transprob[i,j] = ss.norm.pdf(Ylevels2[j], mu_cond, sigma_cond)

    #normalize probabilities to sum to 1
    transprob[i,:] = transprob[i,:] / np.sum(transprob[i,:])

  return transprob

# discretize inflows into nLevels and find transition probabilities between them
nLevels=20
transprob = []
for i, index in enumerate(forward_indices[0:-1]):
  mu = np.array([muY[index],muY[forward_indices[i+1]]]) # log-space mean
  sigma = np.array([sigmaY[index],sigmaY[forward_indices[i+1]]]) # log-space standard deviation
  transprob.append(calcTransProb(mu, sigma, rho, nLevels))

$\color{red}{\text{Make a heatmap of the transition probabilities across the 6 seasons in a 3x2 panel figure in the code chunk below.}}$  
Hint: add `ax=ax[:,-1]` to `fig.colorbar()` so the colorbar spans both rows of figures.

## Question 3. SDP cost functions (40 pts - 20 pts each)

Now modify the two functions $\texttt{calcValueSDP}$ and $\texttt{findBestR}$ from class to optimize operations for this problem with SDP.  

$\texttt{calcValueSDP}$ will loop through all discretized storage levels at time $t$ and inflow levels at time $t-1$ to determine the optimal release conditioned on those two values that maximizes the present + expected future value (average storage). Within each of these nested loops, it will do that by calling $\texttt{findBestR}$ with $\texttt{scipy.optimize.minimize}$. Note, we would like to maximize, not minimize, the value, though. Constraint this optimization so that the reservoir capacity is not exceeded even if the 99th percentile flow is received at stage $t$, and the storage is not negative even if the 1st percentile flow is received at stage $t$

$\texttt{calcValueSDP}$ should take as input the following:

* $S$: 1-D array of discrete storage values representing the states
* $Ylevels1$: 1-D array of discrete, log-space inflow levels in the previous stage, that current releases will be conditioned on for each storage level
* $Ylevels2$: 1-D array of discrete, log-space inflow levels in this stage
* $transprob$: 2-D array of transition probabilities from $Ylevels1$ to $Ylevels2$
* $bounds$: 1-D array of length 2 with the lower and upper bounds on the release
* $FutureExpCost$: 2-D array of future expected costs at each combination of $S$ and $Ylevels1$ that will be added to the cost of the optimal state transition at this stage to compute the present + expected future value

And output:

* $Rbest$: 2-D array of optimal releases as a function of $S$ and $Ylevels1$
* $Cbest$: 2-D array of present + expected future values associated with each combination of $S$ and $Ylevels1$

$\texttt{findBestR}$ should take as input the following:

* $R$: scalar representing the release we are trying to find the optimal value for
* $s$: scalar representing the storage level at which we are trying to find the optimal release
* $y1$: scalar representing the inflow level in the previous time step we are conditioning the release on
* $j$: scalar index representing which of the previous time step's inflow levels we are conditioning on
* $S$: 2-D array of storage states for interpolating the future value
* $Ylevels2$: 1-D array of discrete, log-space inflow levels in this stage
* $transprob$: 2-D array of transition probabilities from $Ylevels1$ to $Ylevels2$
* $FutureExpValue$: 2-D array of future expected costs at each combination of $S$ and $Ylevels1$ that will be added to the cost of the optimal state transition at this stage to compute the present + expected future value

And output:

* $-V$: the expected future value associated with the release $R$ at this storage $s$ and inflow $y1$, negated for minimization

$\color{red}{\text{Complete these functions in the code chunk below.}}$

In [ ]:
######################## SDP Optimization ########################
from scipy.optimize import minimize

def findBestR(R, s, y1, j, S, Ylevels2, transprob, FutureExpValue):


  return -V

def calcValueSDP(S, Ylevels1, Ylevels2, Qguess, transprob, bounds, FutureExpValue):


  return Rbest, Vbest

$\color{red}{\text{You can check that your calcValueSDP function returns the right value with the code below.}}$

In [ ]:
index = backward_indices[0]
SDP_FutureExpValue = np.zeros([nStates, nLevels])
mu = np.array([muY[index-2],muY[index-1]])
sigma = np.array([sigmaY[index-2],sigmaY[index-1]])
Ylevels1 = np.linspace(ss.norm.ppf(0.01,mu[0],sigma[0]),ss.norm.ppf(0.99,mu[0],sigma[0]),nLevels)
Ylevels2 = np.linspace(ss.norm.ppf(0.01,mu[1],sigma[1]),ss.norm.ppf(0.99,mu[1],sigma[1]),nLevels)
R, SDP_FutureExpValue = calcValueSDP(states, Ylevels1, Ylevels2, meanQ[index-1], transprob[index-1], bounds, SDP_FutureExpValue)

print("R.mean() should equal [{}]. Yours equals [{}]".format(12.52, R.mean()))
print("SDP_FutureExpValue.mean() should equal [{}]. Yours equals [{}]".format(128.82, SDP_FutureExpValue.mean()))

##Question 4: SDP optimization (10 pts)

Now run backward-moving SDP with the code chunk below, which uses the functions you wrote above to solve for the optimal release from each combination of 20 storage levels and 20 previous inflow levels. It stops when the average difference in the release decisions at each state change by less than $100\%$ in a given stage from one cycle through all seasons to the next. $\color{red}{\text{The code below is complete.}}$

In [ ]:
# use same stages, states, indices and bounds as for DDP
# initialize matrices with values of each state at each stage
# and optimal releases to make from each state at each stage
SDP_values = np.empty([nStates, nLevels, nStages])
SDP_release_policy = np.empty([nStates, nLevels, nStages])+Rmax

# initialize FutureValue at 0 for all states; will update as we move backwards
SDP_FutureExpValue = np.zeros([nStates, nLevels])

# begin backward-moving SDP
tolerance = 100 # average % difference in optimal releases from one cycle to the next at which to stop looping
avgPctDiff = np.inf
cycle = 0 # number of times cycling through all seasons
while loop:
  count = 0
  cycle += 1
  print(cycle)
  for index in backward_indices[0:-1]:
    mu = np.array([muY[index-2],muY[index-1]])
    sigma = np.array([sigmaY[index-2],sigmaY[index-1]])
    Ylevels1 = np.linspace(ss.norm.ppf(0.01,mu[0],sigma[0]),ss.norm.ppf(0.99,mu[0],sigma[0]),nLevels)
    Ylevels2 = np.linspace(ss.norm.ppf(0.01,mu[1],sigma[1]),ss.norm.ppf(0.99,mu[1],sigma[1]),nLevels)

    # find optimal release and value of each state in this stage
    # states (storage targets) are for next period (index) while inflows and releases are for this period (index-1)
    R, SDP_FutureExpValue = calcValueSDP(states, Ylevels1, Ylevels2, meanQ[index-1], transprob[index-1], bounds, SDP_FutureExpValue)

    # find average % difference in optimal releases at this stage compared to the last loop
    if cycle > 1:
      avgPctDiff = np.ma.masked_invalid(np.abs(R - SDP_release_policy[:,:,index-1])*100/SDP_release_policy[:,:,index-1]).mean()
      if avgPctDiff < tolerance:
        count += 1

    # update best releases and value of each state
    SDP_values[:,:,index] = SDP_FutureExpValue
    SDP_release_policy[:,:,index-1] = R

  # stop loop if average % change in optimal decisions across all iterations < tolerance
  if count == len(backward_indices[0:-1]):
    break

$\color{red}{\text{Make a heatmap of the releases as a function of the current storage and previous inflow across the 6 seasons in a 3x2 panel figure.}}$  
Again, add `ax=ax[:,-1]` to `fig.colorbar()` to make the colorbar span both rows.

## Question 5: Simulation of DDP and SDP (5 pts)

Now simulate operations with your DDP and SDP policies for 50 years of 6 seasons. If the storage capacity is going to be exceeded, the code below will increase the release to that which prevents it from being exceeded. This may violate the max release constraint. The number of these Rmax_violations is also calculated.

$\color{red}{\text{Edit getSimValue below to return the average simulated storage level of the operations each year.}}$

In [ ]:
######################## Simulation ########################

# initialize storages and releases for simulation of 50 years of 3 seasons with NLP and DP policies
nYears = 50
nSeasons = 6

class Solution():
  # initialize Solution class with certain attributes for DP vs. NLP solution
  def __init__(self):
    self.simS = np.zeros([nYears,nSeasons])
    self.simR = np.zeros([nYears,nSeasons])
    self.value = np.zeros([nYears])
    self.prescribedR = None
    self.Rmax_violations = 0

  # method of Solution class to calculate simulated R and S
  def getSimStates(self, Q, year, season):
    # adjust prescribed release if not physically possible
    # R = min(prescribedR, simS + Q) prevents it from releasing more water than is available
    # max(simS + Q - K, R) prevents storage capacity from being exceeded
    self.simR[year,season] = max(self.simS[year,season] + Q - K,
                                 min(self.prescribedR, self.simS[year,season] + Q))

    # find number of violations of maxR and Qirr
    if self.simR[year,season] > Rmax:
      self.Rmax_violations += 1

    # calculate new storage
    if season != (nSeasons-1): # storage in next season of same year
      self.simS[year,season+1] = self.simS[year,season] + Q - self.simR[year,season]
    elif year != (nYears-1): # storage in season 1 of next year
      self.simS[year+1,0] = self.simS[year,season] + Q - self.simR[year,season]

  # method of Solution class to calculate value (average storage) over simulation
  def getSimValue(self, year):


Now run the code below to compute the value of DDP_avg, DDP_99 and SDP over the 50-year simulation, and compute its average value and number of Rmax violations. Each policy is initiated at a storage of 100 (half-full). $\color{red}{\text{The code below is complete.}}$

In [ ]:
from scipy.interpolate import RegularGridInterpolator as interp2d

# create objects of Solution class for NLP and DP solutions
DDP_avg = Solution()
DDP_99 = Solution()
SDP = Solution()

# start half full
DDP_avg.simS[0,0] = 100
DDP_99.simS[0,0] = 100
SDP.simS[0,0] = 100

# vector of standard normal random variables for flow simulation
Z = np.zeros([nYears*nSeasons+1])

# generate prior season's random normal inflow
seed = 0
np.random.seed(seed)
Z[seed] = ss.norm.rvs(0,1,1)[0]
Qpast = np.exp(Z[seed]*sigmaY[-1] + muY[-1])

# simulate operations over 50 years of 4 seasons
for year in range(nYears):
  for season in range(nSeasons):
    # generate inflow (set a seed to make it reproducible)
    seed += 1
    np.random.seed(seed)
    # generate flow conditional on previous season's flow and transform to real-space
    Z[seed] = rho*(Z[seed-1]) + ss.norm.rvs(0,1,1)[0]*np.sqrt(1-rho**2)
    Qnow = np.exp(Z[seed]*sigmaY[season] + muY[season])

    # find DDP releases from its policy: interpolate release between nearest storages
    DDP_avg.prescribedR = np.interp(DDP_avg.simS[year,season],states,DDP_release_policy[:,season])
    DDP_99.prescribedR = np.interp(DDP_99.simS[year,season],states,DDP_release_policy_99[:,season])

    # find SDP releases from its policy: interpolate release between nearest storages and flows
    Ylevels1 = np.linspace(ss.norm.ppf(0.01,muY[season-1],sigmaY[season-1]),ss.norm.ppf(0.99,muY[season-1],sigmaY[season-1]),nLevels)
    f = interp2d((states, Ylevels1), SDP_release_policy[:,:,season])
    if np.log(Qpast) <= Ylevels1[0]: # interpolate between storages at lowest flow level
      SDP.prescribedR = np.interp(SDP.simS[year,season],states,SDP_release_policy[:,0,season])
    elif np.log(Qpast) >= Ylevels1[-1]: # interpolate between storages at highest flow level
      SDP.prescribedR = np.interp(SDP.simS[year,season],states,SDP_release_policy[:,-1,season])
    else: # interpolate over 2-D grid
      SDP.prescribedR = f(np.array([SDP.simS[year,season],np.log(Qpast)]))[0]

    # find actual release (what is physically possible) and calculate storage from mass balance
    DDP_avg.getSimStates(Qnow, year, season)
    DDP_99.getSimStates(Qnow, year, season)
    SDP.getSimStates(Qnow, year, season)

    # update past flow for next season's calculation
    Qpast = Qnow

  # calculate total cost (squared deviations from targets) in simulated year
  DDP_avg.getSimValue(year)
  DDP_99.getSimValue(year)
  SDP.getSimValue(year)

## Question 6. Plot CDFs of average simulated storage (10 pts)

$\color{red}{\text{Use the code chunk below to make a CDF of the average value from DDP_avg, DDP_99 and SDP across the 50-year simulation. What do you observe?}}$

## Question 7. Make barplot of number of simulated Rmax violations (10 pts)

$\color{red}{\text{Now make a bar plot of the number of release violations from DDP_avg, DDP_99, and SDP over the 50-years. What do you notice?}}$  
$\color{red}{\text{Based on the results in parts (d) and (e), which method would you recommend using to optimize reservoir operations and why?}}$